# MLP Classiifers DEF

  - 3. Syndigo Mapping MachineLearning

In [1]:
def unique_no_nan(x):
    return x.dropna().unique()

In [2]:
def constrcut_dic():
    key = unique_no_nan(syndigo_mapped['Level 1'])
    value = list( map(lambda x : re.sub('&|/', '', x), key))
    value = list( map(lambda x : re.sub(' +', '_', x), value))
    value = list(map(str.lower, value))
    #value = ['-'.join(['tfidf'+ str]), value]

    # print(f'key: {key}') 
    # print(f'value: {value}')
    return dict(zip(key, value)) 

In [3]:
def prepare_data_1():
    # Reading PIMMART data
    pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
        pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)
    
    # Reading Syndigo 259K data
    synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
        synd_ALL[i] = synd_ALL[i].astype(np.float64)
    
    # Stripping spaces from all columns
    df_obj = synd_ALL.select_dtypes(['object'])
    synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    syndigo_mapped = synd_ALL
    pimmart = pim_gtin_mapped
    syndigo_mapped.drop_duplicates('GTIN', inplace = True)
    syndigo_mapped['ITEM_SUBCOM_text'] = \
    (syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()
    #syndigo_mapped['Level 1'].value_counts()
    print(f'syndigo_mapped.shape {syndigo_mapped.shape}') 
    #print(f'syndigo_mapped.info() {syndigo_mapped.info()}') 
    #print(f'syndigo_mapped.head() {syndigo_mapped.head()}') 
    #print(f'syndigo_mapped.columns {syndigo_mapped.columns}') 
    
    print(f"syndigo_mapped['Level 1'].value_counts() {syndigo_mapped['Level 1'].value_counts(dropna = False)}")    
    return syndigo_mapped, pim_gtin_mapped

In [4]:
def prepare_data_2(level_1):
    global   k_neighbors
    subset_df = syndigo_mapped[syndigo_mapped['Level 1'] == level_1]

    series =  subset_df['Level 2'].value_counts(ascending=True, dropna = False)
    k_neighbors = 5
    if min(series) < 10:
        k_neighbors = 3 
        drop_cut_off = 10
        subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series  < drop_cut_off ].index.tolist())].index) 
    print (f'Min cnt of Level 2 for class 1  {level_1}  is {min(series)} and k_neighbors is {k_neighbors}')
      
    x_subset_w2v = np.array(list(subset_df.ITEM_SUBCOM_text.apply(lambda x: get_item_vector(x.split(' ')))))                                            
    level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
    id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
    y_subset_w2v  = subset_df['Level 2'].fillna('Other').map(level2_id_map)
    
    return  x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map

In [5]:
#Build item vectors
def get_item_vector(item_vocab):
    vect = np.zeros_like(model.get_vector('chips'))
    for word in item_vocab:
        if word in model:
            vect += model.get_vector(word)
    return vect#/max(1,len(item_vocab))

In [6]:
def proc_predict(level_1):

     mlp = MLPClassifier(hidden_layer_sizes=(200,), activation = 'relu', learning_rate= 'adaptive', early_stopping= True)
     #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
     print(f"Training - w2v + MLP \"{level_1}\"")
     warnings.filterwarnings("ignore", category=UserWarning)
     mlp.fit(X_train, y_train)
     print(' Display MLP test  metrics')
     preds_test_mlp = mlp.predict(X_test)
     preds_train_mlp = mlp.predict(X_train)
    
     print(classification_report(y_test, preds_test_mlp,labels = mlp.classes_, target_names = [id2_level_map[i] for i in mlp.classes_]))      
     return 

In [7]:
def get_smp_strgy(dic_y_train, adj_ratio): 
    """
        get_smp_strgy  :  get sampling_strategy for SMOTE
    """
    smp_strgy = {} 
    
    adj_cnt =  int(max(dic_y_train.values()) * adj_ratio)
    for key, values in dic_y_train.items():
        if values < adj_cnt:
           smp_strgy[key] = adj_cnt
        else: 
            smp_strgy [key] = values
            
    return smp_strgy

In [16]:
def proc_smote(level_1): 
    print(f' smote level_1 : {level_1}')
    X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= smp_strgy, k_neighbors =   k_neighbors).fit_resample(X_train, y_train)
                                                                                             
    print('Before Resampling:\n',pd.Series(y_train).value_counts())
    print('After Resampling:\n',pd.Series(y_resampled).value_counts())
    mlp_smote = MLPClassifier(hidden_layer_sizes=(200,), activation = 'relu', learning_rate= 'adaptive', early_stopping= True)
    #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
    print(f"Training - w2v + MLP \"{level_1}\"")
    warnings.filterwarnings("ignore", category=UserWarning)
    mlp_smote.fit(X_resampled, y_resampled)
    
    print(' Display MLP + SMOTE test  metrics')
    preds_test_mlp_smote = mlp_smote.predict(X_test)
    preds_train_mlp_smote = mlp_smote.predict(X_resampled)
    print(classification_report(y_test, preds_test_mlp_smote,labels = mlp_smote.classes_, target_names = [id2_level_map[i] for i in mlp_smote.classes_]))
    return

In [9]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier,  RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import gensim
from imblearn.over_sampling import SMOTE, ADASYN
import logging

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
import re

In [10]:
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path


In [11]:
syndigo_mapped, pim_gtin_mapped = prepare_data_1()

syndigo_mapped.shape (259085, 24)
syndigo_mapped['Level 1'].value_counts() Food / Beverages                            131593
Health & Beauty                              62259
Beer / Wine / Spirits                        17992
Cleaning & Janitorial                         9084
Livestock & Pet Supplies                      6325
Kitchen & Bathroom                            5268
Home & Venue Decoration                       3924
Toys / Games / Hobbies                        3191
Gardening & Outdoors                          2208
Childcare                                     2111
Office Supplies                               1671
Electronics                                   1499
Apparel                                       1283
Lighting & Fans                               1252
Tobacco Products                              1252
NaN                                           1132
Not classified                                1018
Hardware                                       923
Arts & 

In [12]:
dic_level1= constrcut_dic()


In [17]:
#dic_test={"Hospitality Supplies":"hospitality_spplies", "Tobacco Products":"bacco_products"}

### import word2vec model trained on entire PIMMART data
from gensim.models import Word2Vec, KeyedVectors
#model = KeyedVectors.load_word2vec_format('/dbfs/FileStore/tables/DATA_SCIENCE/w2vmodel_053123_PIM_ALL.bin', binary=True)
model = KeyedVectors.load_word2vec_format(path + 'w2vmodel_053123_PIM_ALL.bin', binary=True)                                        
model.most_similar('chips')
adj_ratio = 0.2
i =0
err_smote_cnt = 0
err_smt_cnt = 0
#N = 5 
#dic_level1= constrcut_dic()
#dic_level1 = dict(list(dic_level1.items())[4: N])

for level_1, filenamee in dic_level1.items():    
    print(f'{i}. Process level 1 = {level_1}')    
    i += 1    
    x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map =  prepare_data_2(level_1)
    if  len ( x_subset_w2v) < 10: 
        len_x_subset_w2v = len(x_subset_w2v)
        print(f'len(x_subset_w2v) = {len_x_subset_w2v}. Not  process len(x_subset_w2v) < 10')
        continue
    try:
       X_train, X_test, y_train, y_test = train_test_split(x_subset_w2v, y_subset_w2v.values, test_size= 0.3, stratify=y_subset_w2v, random_state=42)

       proc_predict(level_1)
       dic_y_train = dict(pd.Series(y_train).value_counts())
       smp_strgy = get_smp_strgy(dic_y_train, adj_ratio)
      
       try:
          proc_smote(level_1)
       except Exception as smote_ve:
          err_smote_cnt += 1 
          logging.exception("number of smote error= %d.   Exception occurred for class_1 %s : %s", err_smote_cnt , level_1, str(smote_ve))
          continue
    except Exception as smp_ve:
       err_smp_cnt += 1 
       len_x_subset_w2v  = len ( x_subset_w2v)
       logging.exception("number of smaple error %d. len ( x_subset_w2v) = %d.  Exception occurred: %s", err_smp_cnt , len_x_subset_w2v,  str(smp_ve))

0. Process level 1 = Food / Beverages
Min cnt of Level 2 for class 1  Food / Beverages  is 309 and k_neighbors is 5
Training - w2v + MLP "Food / Beverages"
 Display MLP test  metrics
                                                          precision    recall  f1-score   support

                                    Dairy & Egg Products       0.88      0.94      0.91      2705
                                                 Grocery       0.94      0.96      0.95     22459
                                               Beverages       0.96      0.95      0.96      6372
Meat / Poultry / Seafood / Meat Substitutes (Perishable)       0.68      0.76      0.72      1193
                                            Frozen Foods       0.88      0.89      0.89      4074
                                           Bakery / Deli       0.67      0.41      0.51      1600
                                     Baby & Toddler Food       0.86      0.93      0.89       487
                                

 Display MLP + SMOTE test  metrics
                             precision    recall  f1-score   support

                   Bathroom       0.84      0.82      0.83        51
                    Kitchen       0.99      1.00      1.00      1515
                    Faucets       1.00      0.75      0.86         8
Utility Sinks & Accessories       1.00      0.29      0.44         7

                   accuracy                           0.99      1581
                  macro avg       0.96      0.71      0.78      1581
               weighted avg       0.99      0.99      0.99      1581

4. Process level 1 = Cleaning & Janitorial
Min cnt of Level 2 for class 1  Cleaning & Janitorial  is 2 and k_neighbors is 3
Training - w2v + MLP "Cleaning & Janitorial"
 Display MLP test  metrics
                             precision    recall  f1-score   support

             Air Fresheners       0.97      0.96      0.96       532
         Cleaning Chemicals       0.95      0.98      0.96      1345
      

 Display MLP + SMOTE test  metrics
                             precision    recall  f1-score   support

            Games & Puzzles       0.93      0.78      0.85       162
                Toy Figures       0.75      0.85      0.80       342
     Remote & Radio Control       0.33      0.36      0.35        11
          Baby Toys & Games       0.65      0.49      0.56       112
Role Playing & Pretend Play       0.38      0.35      0.36        49
    Construction & Modeling       0.83      0.76      0.80       118
               Toy Vehicles       0.74      0.77      0.75        77
       Outdoor Toys & Games       0.59      0.65      0.62        57
        Sports Toys & Games       0.32      0.44      0.37        25
           Nature / Science       0.00      0.00      0.00         4

                   accuracy                           0.72       957
                  macro avg       0.55      0.55      0.55       957
               weighted avg       0.73      0.72      0.72       9

Before Resampling:
 0     230
5      44
2      34
4      29
1      27
6      20
3      20
9      16
7      10
8       8
10      7
dtype: int64
After Resampling:
 0     230
4      46
7      46
6      46
1      46
2      46
5      46
3      46
9      46
8      46
10     46
dtype: int64
Training - w2v + MLP "Sports & Outdoor Recreation Equipment"
 Display MLP + SMOTE test  metrics
                           precision    recall  f1-score   support

     Water Bottles & Caps       0.90      0.97      0.93        98
         Camping & Hiking       0.80      0.36      0.50        11
       Exercise / Fitness       0.67      0.86      0.75        14
                     Golf       0.80      0.89      0.84         9
Water Sports & Recreation       0.70      0.54      0.61        13
                  Cycling       1.00      0.84      0.91        19
        Hunting & Fishing       0.88      0.78      0.82         9
           Racquet Sports       0.67      0.50      0.57         4
               

ERROR:root:number of smote error= 1.   Exception occurred for class_1 Plumbing & Water Service : The target 'y' needs to have more than 1 class. Got 1 class instead
Traceback (most recent call last):
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_3556\2789689793.py", line 32, in <module>
    proc_smote(level_1)
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_3556\1348509616.py", line 3, in proc_smote
    X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= smp_strgy, k_neighbors =   k_neighbors).fit_resample(X_train, y_train)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 203, in fit_resample
    return super().fit_resample(X, y)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 84, in fit_resample
    self.sampling_strategy_ = check_sampling_strategy(
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\utils\_validation.py", line 514, in check_samp

...

 Display MLP + SMOTE test  metrics
                               precision    recall  f1-score   support

                    Computers       0.78      0.85      0.81        74
        Video Games & Systems       0.69      0.75      0.72        32
      Cables & Cable Adapters       0.76      0.65      0.70        34
                Audio & Video       0.88      0.76      0.81        78
  Telephones & Communications       0.58      0.61      0.59        49
             Power Management       0.89      0.90      0.90       125
           Office Electronics       0.55      0.60      0.57        10
         Cameras & Camcorders       0.78      0.50      0.61        14
                 Data Storage       0.79      0.83      0.81        23
Security Systems & Components       0.50      1.00      0.67         4

                     accuracy                           0.78       443
                    macro avg       0.72      0.74      0.72       443
                 weighted avg       0.79

ERROR:root:number of smote error= 3.   Exception occurred for class_1 Books & Videos : The target 'y' needs to have more than 1 class. Got 1 class instead
Traceback (most recent call last):
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_3556\2789689793.py", line 32, in <module>
    proc_smote(level_1)
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_3556\1348509616.py", line 3, in proc_smote
    X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= smp_strgy, k_neighbors =   k_neighbors).fit_resample(X_train, y_train)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 203, in fit_resample
    return super().fit_resample(X, y)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 84, in fit_resample
    self.sampling_strategy_ = check_sampling_strategy(
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\utils\_validation.py", line 514, in check_sampling_strat

 Display MLP + SMOTE test  metrics
                              precision    recall  f1-score   support

                   Fasteners       0.87      0.92      0.89       168
Adhesives & Sealing Products       0.93      0.81      0.86        47
                  Lubricants       0.75      1.00      0.86         3
             Keys & Padlocks       0.72      0.76      0.74        17
           Builders Hardware       0.62      0.65      0.63        20
             General Purpose       0.14      0.14      0.14         7
                Chain & Rope       1.00      0.25      0.40         8

                    accuracy                           0.83       270
                   macro avg       0.72      0.65      0.65       270
                weighted avg       0.84      0.83      0.82       270

22. Process level 1 = Books & Videos
Min cnt of Level 2 for class 1  Books & Videos  is 1 and k_neighbors is 3
Training - w2v + MLP "Books & Videos"
 Display MLP test  metrics
               p

ERROR:root:number of smote error= 4.   Exception occurred for class_1 Power Sports : The target 'y' needs to have more than 1 class. Got 1 class instead
Traceback (most recent call last):
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_3556\2789689793.py", line 32, in <module>
    proc_smote(level_1)
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_3556\1348509616.py", line 3, in proc_smote
    X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= smp_strgy, k_neighbors =   k_neighbors).fit_resample(X_train, y_train)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 203, in fit_resample
    return super().fit_resample(X, y)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 84, in fit_resample
    self.sampling_strategy_ = check_sampling_strategy(
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\utils\_validation.py", line 514, in check_sampling_strateg

 Display MLP test  metrics
                     precision    recall  f1-score   support

             Office       1.00      0.80      0.89         5
Childrens Furniture       0.80      1.00      0.89         4

           accuracy                           0.89         9
          macro avg       0.90      0.90      0.89         9
       weighted avg       0.91      0.89      0.89         9

 smote level_1 : Furniture
Before Resampling:
 0    11
1     9
dtype: int64
After Resampling:
 0    11
1     9
dtype: int64
Training - w2v + MLP "Furniture"
 Display MLP + SMOTE test  metrics
                     precision    recall  f1-score   support

             Office       1.00      1.00      1.00         5
Childrens Furniture       1.00      1.00      1.00         4

           accuracy                           1.00         9
          macro avg       1.00      1.00      1.00         9
       weighted avg       1.00      1.00      1.00         9

28. Process level 1 = Power Sports
Min cnt 